## Sequential Version of the Code 

In [15]:
# Import the necessary libraries
import os
import cv2
import numpy as np
import time

In [16]:
# Shared variables
processed_image_count = 0 
row_processed = 0  

In [ ]:
# Method to filter on images
def custom_filter_operation(image):

    global row_processed

    Start_time = time.time()
    # Applying Guassian Blur
    blurred_image = cv2.GaussianBlur(image, (3,3), 0)
    print(f"Time taken to apply Gaussian Blur to an image: {time.time() - Start_time}s")
        
    Start_time = time.time()
    # Computing the elevation map using Sobel operator for edge detection
    sobel_x = cv2.Sobel(blurred_image, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blurred_image, cv2.CV_64F, 0, 1, ksize=3)
        
    # Combining the gradients to obtain the magnitude of the gradient
    magnitude_gradient = cv2.magnitude(sobel_x, sobel_y)
    print(f"Time taken to apply Sobel filter to an image: {time.time() - Start_time}s")
    
    Start_time = time.time()
    # Normalizing the image to a range between 0 and 1 (floating-point operations)
    image = magnitude_gradient.astype(np.float32) / 255.0
    print(f"Time taken to normalize the 1 image: {time.time() - Start_time}s")

    # Getting the dimensions of the image
    height, width = image.shape

    # Creating an output image initialized to zero
    output_image = np.zeros((height, width), dtype=np.float32)

    # Custom filter (2D array) for processing
    filter_kernel = np.array([[0.1, 0.2, 0.1],
                              [0.2, 0.1, 0.2],
                              [0.1, 0.2, 0.1],], dtype=np.float32)
    filter_size = filter_kernel.shape[0]

    # Nested loops to apply the filter to the image (floating-point operation)
    Start_time = time.time()
    for i in range(height):
        for j in range(width):
            # Accumulating the weighted sum for the filter
            weighted_sum = 0.0
            for k in range(filter_size):
                for l in range(filter_size):
                    # Calculating the position in the original image
                    row = i + k - filter_size // 2
                    col = j + l - filter_size // 2
                    
                    # Checking if indices are within bounds (Conditional loop)
                    if 0 <= row < height and 0 <= col < width:
                        weighted_sum += image[row, col] * filter_kernel[k, l]

            # Updating the rows processed             
            row_processed += 1

            # Setting the pixel value in the output image 
            output_image[i, j] = weighted_sum

    print(f"Time taken to apply filter to 1 image (Nested Loop): {time.time() - Start_time}s")

    Start_time = time.time()    
    # Clipping the output to stay in the range [0, 1]
    output_image = np.clip(output_image, 0.0, 1.0)
    print(f"Time taken to Clip the output to stay in the range [0, 1]: {time.time() - Start_time}s")

    Start_time = time.time()
    # Converting the processed image back to 8-bit (0-255)
    processed_image = (output_image * 255).astype(np.uint8)
    print(f"Time taken to Convert the image back to 8-bit (0-255): {time.time() - Start_time}s")

    return processed_image

In [18]:
# Folder containing the images
input_folder = 'cars'  # Replace with the path to your folder
output_folder = 'cars_filtered'  # Replace with the path to save filtered images

# Creating the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

In [19]:
Start_time_final = time.time()

# Looping through all the files in the folder
for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):  # Checking for image file types
        
        global processed_image_count

        # Reading the image in grayscale
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        if image is None:
            print(f"Could not open or find the image: {image_path}")

        # Applying the custom filter operation
        Start_time = time.time()
        filtered_image = custom_filter_operation(image)
        processing_time = time.time() - Start_time
        print(f"Total Time taken to apply custom filter to the image: {processing_time}s")

        # Updating images processed 
        processed_image_count += 1

        # Saving the processed image to the output folder
        output_path = os.path.join(output_folder, f'filtered_{filename}')
        cv2.imwrite(output_path, filtered_image)

        print(f'Processed and saved: {output_path}')
        print("\n")
        
# Printing final values        
print(f"Total number of processed images: {processed_image_count}")
print(f"Total number of prcoessed rows: {row_processed}")        
print(f"Total time taken by the execution of this code sequentially for all images: {time.time() - Start_time_final}s")

Time taken to apply Gaussian Blur to an image: 0.0010068416595458984s
Time taken to apply Sobel filter to an image: 0.003005504608154297s
Time taken to normalize the 1 image: 0.0005090236663818359s
Time taken to apply filter to 1 image (Nested Loop): 1.7924039363861084s
Time taken to Clip the output to stay in the range [0, 1]: 0.0005021095275878906s
Time taken to Convert the image back to 8-bit (0-255): 0.0004978179931640625s
Total Time taken to apply custom filter to the image: 1.7994296550750732s
Processed and saved: cars_filtered\filtered_carsgraz_001.bmp


Time taken to apply Gaussian Blur to an image: 0.0s
Time taken to apply Sobel filter to an image: 0.002506256103515625s
Time taken to normalize the 1 image: 0.0009999275207519531s
Time taken to apply filter to 1 image (Nested Loop): 1.8040239810943604s
Time taken to Clip the output to stay in the range [0, 1]: 0.0004982948303222656s
Time taken to Convert the image back to 8-bit (0-255): 0.0s
Total Time taken to apply custom filt

In [20]:
from multiprocessing import cpu_count

logical_cores = cpu_count()
print(f"Number of logical cores: {logical_cores}")

Number of logical cores: 16
